## Writing CNNs from Scratch in Python
### Importing the libraries
Let's start by importing the required libraries and defining some variables:

In [1]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("This is the using device: ", device)

/home/huawei/anaconda3/envs/IACT/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### Dataset Loading

In [2]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])

# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

100%|██████████| 170498071/170498071 [00:29<00:00, 5738059.15it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### CNNs From Scratch

#### Formula
The amount of patameter = ((Kernal_size_width x kernal_size_height X input_channel_from_first_layer) + 1) X output_channel_from_first_layer + ((Kernal_size_width x kernal_size_height X input_channel_from_second_layer) + 1) X output_channel_from_second_layer +...+fully layer

fully layer followes general neural network

In [26]:
import torch
import torch.nn as nn

num_classes = 10

# Creating a CNN class
class ConvNeuralNet(nn.Module):
    #  Determine what layers and their order in CNN object
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2) 
        # the size of output:(64x5x5)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        print("This is the original shape:", x.shape)
        out = self.conv_layer1(x)
        print("After 1 layer shape:", out.shape)
        out = self.conv_layer2(out)
        print("After 2 layer shape:", out.shape)
        out = self.max_pool1(out)
        print("After pool1 shape:", out.shape)
        out = self.conv_layer3(out)
        print("After 3 layer shape:", out.shape)
        out = self.conv_layer4(out)
        print("After 4 layer shape:", out.shape)
        out = self.max_pool2(out)
        print("After pool2 shape:", out.shape)

        out = out.reshape(1,-1)
        print("After re-shape:", out.shape)
        out = self.fc1(out)
        print("After fully-layer 1 shape:", out.shape)
        out = self.relu1(out)
        out = self.fc2(out)
        print("After fully-layer 2 shape:", out.shape)
        return out
    
model = ConvNeuralNet(num_classes)    
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("This is the trainable parameter:",trainable_params)

# tensor_32x32_int = torch.randint(0, 10, (3, 32, 32))
# output=model(tensor_32x32_int.float())
# print(output.shape)


This is the trainable parameter: 271786


### Setting Hyperparameters

In [4]:
model = ConvNeuralNet(num_classes)

if torch.cuda.is_available():
    model.cuda()

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)

### Training

In [5]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
    #Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


Epoch [1/20], Loss: 1.4448
Epoch [2/20], Loss: 1.4486
Epoch [3/20], Loss: 1.1496
Epoch [4/20], Loss: 0.6814
Epoch [5/20], Loss: 0.8725
Epoch [6/20], Loss: 1.1217
Epoch [7/20], Loss: 0.9020
Epoch [8/20], Loss: 1.5773
Epoch [9/20], Loss: 0.8536
Epoch [10/20], Loss: 0.5001
Epoch [11/20], Loss: 1.5807
Epoch [12/20], Loss: 0.9483
Epoch [13/20], Loss: 1.1236
Epoch [14/20], Loss: 0.6921
Epoch [15/20], Loss: 0.8398
Epoch [16/20], Loss: 0.6422
Epoch [17/20], Loss: 0.7614
Epoch [18/20], Loss: 0.7288
Epoch [19/20], Loss: 0.4437
Epoch [20/20], Loss: 0.1868


### Testing

In [6]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))



Accuracy of the network on the 50000 train images: 82.506 %


In [19]:
import torch

# Generate a 32x32 tensor with random integer values between 0 and 9
tensor_32x32_int = torch.randint(0, 10, (3, 32, 32))

print(tensor_32x32_int.shape)

conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
output=conv_layer1(tensor_32x32_int.float())
print(output.shape)

torch.Size([3, 32, 32])
torch.Size([32, 30, 30])


## total

In [22]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("This is the using device: ", device)

# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])

# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

# Creating a CNN class
class ConvNeuralNet(nn.Module):
    #  Determine what layers and their order in CNN object
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2) 
        # the size of output:(64x5x5)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        print("This is the original shape:", x.shape)
        out = self.conv_layer1(x)
        print("After 1 layer shape:", out.shape)
        out = self.conv_layer2(out)
        print("After 2 layer shape:", out.shape)
        out = self.max_pool1(out)
        print("After pool1 shape:", out.shape)
        out = self.conv_layer3(out)
        print("After 3 layer shape:", out.shape)
        out = self.conv_layer4(out)
        print("After 4 layer shape:", out.shape)
        out = self.max_pool2(out)
        print("After pool2 shape:", out.shape)

        out = out.reshape(out.size(0), -1)
        print("After re-shape:", out.shape)
        out = self.fc1(out)
        print("After fully-layer 1 shape:", out.shape)
        out = self.relu1(out)
        out = self.fc2(out)
        print("After fully-layer 2 shape:", out.shape)
        return out
    
# model = ConvNeuralNet(num_classes)    
# # trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# # print("This is the trainable parameter:",trainable_params)

# tensor_32x32_int = torch.randint(0, 10, (3, 32, 32))
# output=model(tensor_32x32_int.float())
# print(output.shape)


model = ConvNeuralNet(num_classes)

if torch.cuda.is_available():
    model.cuda()

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)


# We use the pre-defined number of epochs to determine how many iterations to train the network on

    #Load in the data in batches using the train_loader object
for i, (images, labels) in enumerate(train_loader):  
    if i==1:
    # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



This is the using device:  cuda
Files already downloaded and verified
Files already downloaded and verified
This is the original shape: torch.Size([64, 3, 32, 32])
After 1 layer shape: torch.Size([64, 32, 30, 30])
After 2 layer shape: torch.Size([64, 32, 28, 28])
After pool1 shape: torch.Size([64, 32, 14, 14])
After 3 layer shape: torch.Size([64, 64, 12, 12])
After 4 layer shape: torch.Size([64, 64, 10, 10])
After pool2 shape: torch.Size([64, 64, 5, 5])
After re-shape: torch.Size([64, 1600])
After fully-layer 1 shape: torch.Size([64, 128])
After fully-layer 2 shape: torch.Size([64, 10])
